# FaIR Harmonized
### Erica Simon, 02/05/24
## Purpose: input harmonized historical and projected emissions trajectories into FaIR
- Emissions span 1750 to 2100
- For now, we will not run probabilistic projections


## Imports

In [115]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pooch

from fair import FAIR
from fair.interface import fill, initialise
from fair.io import read_properties

## Create instance

In [116]:
f = FAIR()

## Define time horizon

In [117]:
f.define_time(1750, 2101, 1)

## Define scenarios

In [118]:
f.define_scenarios(['s1'])

## Define configs

In [119]:
f.define_configs(['c1'])

## Define species & properties

In [120]:
species_configs = pooch.retrieve(
    url = 'https://raw.githubusercontent.com/OMS-NetZero/FAIR/master/examples/data/species_configs_properties_calibration1.2.0.csv',
    known_hash = '4f9d25b21d632cf35953842a627db4b1612b465f68b2a824afff017a7390053b',
)

In [121]:
species, properties = read_properties(filename=species_configs)

# removed in examples from fair-calibrate repo, specifically because 'future projections aren't available'
species.remove("Halon-1202")
species.remove("NOx aviation")

f.define_species(species, properties)

## Create data arrays

In [122]:
# create empty emission and temp arrays
f.allocate()   

## Fill data arrays
- Emissions & forcings
- Climate configs
- Species configs
- Initial conditions

### Emissions

In [123]:
df_emis = pd.read_csv('~/outputs/GCAM_harm.csv')
df_emis.head()

,model,scenario,region,Variable,unit,1750,1751,1752,1753,1754,...,2091,2092,2093,2094,2095,2096,2097,2098,2099,2100
0,GCAM 6.0 NGFS,Below 2 C,World,Emissions|BC,Mt BC/yr,2.096766,2.071972,2.067178,2.070382,2.098586,...,1.908504,1.909592,1.910681,1.911769,1.912858,1.913896,1.914934,1.915973,1.917011,1.918049
1,GCAM 6.0 NGFS,Below 2 C,World,Emissions|C2F6,kt C2F6/yr,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.908171,0.903291,0.898392,0.893474,0.888537,0.884270,0.879985,0.875684,0.871367,0.867032
2,GCAM 6.0 NGFS,Below 2 C,World,Emissions|CF4,kt CF4/yr,0.000000,0.000000,0.000000,0.000000,0.000000,...,2.193012,2.099410,2.006379,1.913919,1.822028,1.750974,1.680355,1.610170,1.540421,1.471106
3,GCAM 6.0 NGFS,Below 2 C,World,Emissions|CH4,Mt CH4/yr,38.246272,38.106144,38.070994,38.237775,38.585534,...,246.543212,246.884856,247.226500,247.568144,247.909788,248.005228,248.100668,248.196108,248.291548,248.386987
4,GCAM 6.0 NGFS,Below 2 C,World,Emissions|CO,Mt CO/yr,348.454973,343.853051,342.423860,343.601163,349.741197,...,319.248994,319.258361,319.267626,319.276792,319.285857,319.436284,319.586502,319.736512,319.886312,320.035904


In [124]:
# remove 'Emissions|' from variable name
for i in range(len(df_emis.Variable.values)): 
    df_emis.Variable.values[i] = df_emis.Variable.values[i][10:]

Fill `f.emissions` with values from `df_emis`

In [125]:
for var in df_emis['Variable']:
    if var in f.species:
        emis = df_emis[df_emis['Variable'] == var].values[0][5:].reshape(351,1)
        fill(f.emissions, emis, scenario='s1', specie=var)

We also need to check if any relevant species are not included in this dataset, excluding ones that are calculated from other values.

In [153]:
for element in f.species:
    if element not in df_emis['Variable'].values:
        if f.properties[element]['input_mode'] != 'calculated':
            print(element)

CHCl3
CH2Cl2
CH3Cl
CH3CCl3
CH3Br
Halon-1211
Halon-1301
Halon-2402
C3F8
c-C4F8
C4F10
C5F12
C6F14
C7F16
C8F18
NF3
SO2F2
HFC-152a
HFC-236fa
HFC-365mfc
HFC-4310mee
Solar
Volcanic


In [127]:
# for spc in missing_species[:-2]:
#     species.remove(spc)

### solar & volcanic forcings

In [128]:
solar_obj = pooch.retrieve(
    url = 'https://raw.githubusercontent.com/chrisroadmap/fair-add-hfc/main/data/solar_erf_timebounds.csv',
    known_hash = 'md5:98f6f4c5309d848fea89803683441acf',
)

In [129]:
volcanic_obj = pooch.retrieve(
    url = 'https://raw.githubusercontent.com/chrisroadmap/fair-calibrate/main/data/forcing/volcanic_ERF_1750-2101_timebounds.csv',
    known_hash = 'md5:c0801f80f70195eb9567dbd70359219d',
)

In [130]:
df_solar = pd.read_csv(solar_obj, index_col="year")
df_volcanic = pd.read_csv(volcanic_obj, index_col="timebounds")

The solar forcing data spans from 1750-2101, but the volcanic has data until 2300, so we need to trim `volcanic_df` to only include 1750-2101.

In [131]:
s_yr_list = list(range(2102, 2301))
df_solar = df_solar.drop(s_yr_list)

In [132]:
fill(f.forcing, df_volcanic, scenario='s1', specie='Volcanic')
fill(f.forcing, df_solar, scenario='s1', specie='Solar')

### Climate configs

In [133]:
fill(f.climate_configs["ocean_heat_transfer"], [1.1, 1.6, 0.9], config='c1')
fill(f.climate_configs["ocean_heat_capacity"], [8, 14, 100], config='c1')
fill(f.climate_configs["deep_ocean_efficacy"], 1.1, config='c1')

### Species configs

In [134]:
# fill with default values
f.fill_species_configs()

### Initial conditions

In [135]:
initialise(f.concentration, f.species_configs["baseline_concentration"])
initialise(f.forcing, 0)
initialise(f.temperature, 0)   # f.temperature refers to temperature anomaly above PI (288 K)
initialise(f.cumulative_emissions, 0)
initialise(f.airborne_emissions, 0)

## Run

In [154]:
f.species

['CO2 FFI',
 'CO2 AFOLU',
 'CO2',
 'CH4',
 'N2O',
 'Sulfur',
 'BC',
 'OC',
 'NH3',
 'NOx',
 'VOC',
 'CO',
 'CHCl3',
 'CH2Cl2',
 'CH3Cl',
 'CH3CCl3',
 'CH3Br',
 'Halon-1211',
 'Halon-1301',
 'Halon-2402',
 'CF4',
 'C2F6',
 'C3F8',
 'c-C4F8',
 'C4F10',
 'C5F12',
 'C6F14',
 'C7F16',
 'C8F18',
 'NF3',
 'SF6',
 'SO2F2',
 'HFC-125',
 'HFC-134a',
 'HFC-143a',
 'HFC-152a',
 'HFC-227ea',
 'HFC-23',
 'HFC-236fa',
 'HFC-245fa',
 'HFC-32',
 'HFC-365mfc',
 'HFC-4310mee',
 'Solar',
 'Volcanic',
 'Aerosol-radiation interactions',
 'Aerosol-cloud interactions',
 'Ozone',
 'Contrails',
 'Light absorbing particles on snow and ice',
 'Stratospheric water vapour',
 'Land use',
 'Equivalent effective stratospheric chlorine']

In [152]:
f.run()

ValueError: CHCl3 contains NaN values in its emissions array, which you are trying to drive the simulation with.